In [1]:
import os

import numpy as np
import pandas as pd

from utils import get_hash
from utils import Featurizer



In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
# Create a featurizer object that converts a phrase into embedding vector
emb_file = os.path.join("munge", "GoogleNews-vectors-negative300.bin")
featurizer = Featurizer(emb_file)

Loading word vectors...
Done! Using 3000000 word vectors from pre-trained word2vec.


In [12]:
"""problem_description = input(prompt="Problem description:")
question = input(prompt="Question:")
answer = input(prompt="Answer")
"""

problem_description = "Two hockey players pass a puck between them on an ice rink. Assume that the ice is very smooth so that there is no friction."
question = "What forces are acting on the puck while the puck is moving on the ice between the two players? Describe the motion of the puck."
answer = "The normal force coming from the ice and the gravitational force. The velocity of the puck is constant because there are no counteracting forces acting on the puck. As stated by the first law of Newton, an object will maintain its state of motion unless acted upon by a force."

pd_hash = get_hash(problem_description)
qu_hash = get_hash(question)

emb = featurizer.doc2vec(answer)

In [4]:
problem_description = "A basketball player is dribbling a basketball (continuously bouncing the ball off the ground)."
question = "Because the ball's velocity is upward while the ball is moving upward and its acceleration is downward, what is happening to the ball's velocity?"
answer = "The ball's velocity is decreasing."

pd_hash = get_hash(problem_description)
qu_hash = get_hash(question)

emb = featurizer.doc2vec(answer)

In [15]:
problem_description = "A rocket is pushing a meteor with constant force. At one moment the rocket runs out of fuel and stops pushing the meteor. Assume that the meteor is far enough away from the sun and the planets to neglect gravity."
question = "How will the meteor move after the rocket stops pushing?"
answer = "The metor will move toward the rocket at a constant velocity based on Newton's 1st law. Since there is no forces acting on the metor except its normal force which is upward."

pd_hash = get_hash(problem_description)
qu_hash = get_hash(question)

emb = featurizer.doc2vec(answer)

In [18]:
# Load landmark embeddings from disk
landmarks = pd.read_csv(os.path.join("munge", "landmarks.txt"), sep="\t")

# Filter on question of concern
qu_land = landmarks[(landmarks['pd_hash'] == pd_hash) & (landmarks['qu_hash'] == qu_hash)].copy()

# Convert embeddings from string to list of floats
qu_land['embedding'] = qu_land['embedding'].apply(lambda x : list(map(float, x.split(','))))

# Compute image distance to all landmarks
qu_land['dist'] = qu_land['embedding'].apply(lambda x : featurizer.cossim_from_emb(emb, np.array(x)))
qu_land['dist'] = np.round(qu_land['dist'], 6)

In [19]:
qu_land.sort_values('dist')

,pd_hash,qu_hash,label,answer,embedding,dist
697,abb3be1545,c9b7df0ef8,1,THE METEOR WILL MOVE WITH A CONSTANT VELOCITY,"[0.07453264508928571, 0.07338169642857142, -0.074462890625, 0.05079868861607143, -0.14442661830357142, 0.018048967633928572, 0.030604771205357144, -0.14972795758928573, 0.12974330357142858, 0.13541085379464285, -0.03752790178571429, -0.13579450334821427, -0.04695347377232143, 0.004098074776785714, -0.0029384068080357145, 0.0037318638392857145, 0.04502650669642857, 0.13619559151785715, 0.012067522321428572, -0.13155691964285715, -0.10728236607142858, -0.002406529017857143, -0.027195521763392856, 0.03695569719587054, -0.025098528180803572, -0.10511997767857142, -0.0518798828125, 0.04143415178571429, 0.04056004115513393, 0.024518694196428572, -0.10443987165178571, -0.14565604073660715, -0.05995396205357143, -0.13014003208705358, -0.10320172991071429, -0.024370465959821428, -0.029994419642857144, 0.024645124162946428, 0.038120814732142856, 0.02313232421875, 0.12608119419642858, -0.05925641741071429, 0.12489536830357142, 0.012974330357142858, -0.03245326450892857, -0.08864702497209821, ...",0.693729
489,abb3be1545,c9b7df0ef8,1,The meteor will move at a constant velocity.,"[0.06971958705357142, 0.06487165178571429, -0.05901227678571429, 0.04682268415178571, -0.13727678571428573, 0.028982979910714284, 0.052211216517857144, -0.13807896205357142, 0.13138253348214285, 0.13942173549107142, -0.04917689732142857, -0.12773786272321427, -0.015598842075892858, -0.00537109375, 0.006304059709821429, -0.005301339285714286, 0.028494698660714284, 0.13228934151785715, 0.03159877232142857, -0.13138253348214285, -0.12074497767857142, -0.022286551339285716, -0.024065290178571428, 0.04472460065569196, -0.05980137416294643, -0.10391671316964286, -0.045741489955357144, 0.04296875, 0.0509185791015625, 0.00714111328125, -0.1009521484375, -0.16201346261160715, -0.085205078125, -0.16683088030133927, -0.08408900669642858, -0.02935791015625, -0.033656529017857144, 0.005580357142857143, 0.030918666294642856, 0.0341796875, 0.10457938058035714, -0.08820452008928571, 0.137451171875, -0.011439732142857142, -0.026210239955357144, -0.07775660923549108, 0.06849888392857142, -0.02429199...",0.703394
225,abb3be1545,c9b7df0ef8,1,The meteor will continue to move at a constant velocity.,"[0.055419921875, 0.0709228515625, -0.06463623046875, 0.0734405517578125, -0.135986328125, 0.0296630859375, 0.067657470703125, -0.1258544921875, 0.134979248046875, 0.1012420654296875, -0.08502197265625, -0.0893096923828125, -0.00559234619140625, 0.02008056640625, -0.01963043212890625, -0.032470703125, 0.033905029296875, 0.12091064453125, 0.0333251953125, -0.127044677734375, -0.12103271484375, -0.016326904296875, -0.00213623046875, 0.05140209197998047, -0.038898468017578125, -0.1088714599609375, -0.0254974365234375, 0.03624725341796875, 0.03701591491699219, 0.00876617431640625, -0.0763092041015625, -0.15946197509765625, -0.086883544921875, -0.17405319213867188, -0.07501983642578125, -0.00359344482421875, -0.024078369140625, 0.009368896484375, 0.0376129150390625, 0.00927734375, 0.1293487548828125, -0.072662353515625, 0.133697509765625, -0.01593017578125, -0.0338592529296875, -0.07340812683105469, 0.0653076171875, -0.0157318115234375, 0.042816162109375, -0.050048828125, -0.013290405273...",0.714653
446,abb3be1545,c9b7df0ef8,3,The meteor will decrease in velocity and will stop moving.,"[0.029161241319444444, 0.06429036458333333, -0.002224392361111111, 0.050577799479166664, -0.11260308159722222, -0.024197048611111112, -0.012559678819444444, -0.09223090277777778, 0.10915120442708333, 0.10111490885416667, -0.018866644965277776, -0.070068359375, -0.0013800726996527778, 0.023098415798611112, -0.03158908420138889, -0.03366131252712674, -0.029669867621527776, 0.09063042534722222, 0.06471082899305555, -0.13093058268229166, -0.10078938802083333, 0.0006510416666666666, -0.05517578125, 0.004681905110677083, -0.035247802734375, -0.03164333767361111, -0.006997002495659722, 0.09149848090277

In [24]:
featurizer.word2vec.most_similar("decreasing")

[('increasing', 0.8234609365463257),
 ('reducing', 0.7438215017318726),
 ('increased', 0.7151682376861572),
 ('decreased', 0.7057266235351562),
 ('diminishing', 0.6953083276748657),
 ('decrease', 0.6931016445159912),
 ('decreases', 0.6622076630592346),
 ('lessening', 0.6620092988014221),
 ('reduced', 0.6618596911430359),
 ('lowering', 0.6226555109024048)]

In [ ]:



    # Get index of nearest neighbor
    nnidx = data['dist'].idxmin()
    
    # Output predicted class and distance
    label = data['label'][nnidx]
    dist = data['dist'][nnidx]
    print("Predicted class: {}".format(label))
    print("Distance: {}".format(dist))